In [1]:
import sys
!{sys.executable} -m pip install opencv-python

In [3]:
import requests
import cv2
from   bs4 import BeautifulSoup

In [5]:
ASCII   = ''.join(chr(int(N))for N in '074 065 051 049 084 077 048 046 112 110 103'.split())

PI_1MIL = requests.get("https://www.angio.net/pi/digits/pi1000000.txt").text
MAGIC   = "036695"
OFFSET  = PI_1MIL.find(MAGIC) + len(MAGIC)

In [4]:
class Container:
    def __init__(self, value: int, next: object= None):
        self.pNext  = next
        self.iValue = value
 
    @property
    def Next(self):
        return self.pNext
    
    @property
    def Value(self):
        return self.iValue

In [6]:
class Stack:
    def __init__(self):
        self.pHead  = None
        self.iCount = 0
        
    def push(self, element: int):
        self.iCount+= 1
        self.pHead  = Container(element, self.pHead)
        
    def pop(self) -> int:
        if not self.iCount:
            return None
        iValue      = self.pHead.Value
        self.pHead  = self.pHead.Next
        self.iCount-= 1
        return iValue
    
    def getSize(self):
        return self.iCount

In [7]:
class Decimal:
    charset = "0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ"

    def __init__(self):
        self.iIndex   = 0
        self.Rem      = Stack()
        
    def convertTo(self, number: int, base: int) -> str:
        self.Charset  = self.__class__.charset[:base]
        while number != 0:
            self.Rem.push(number % base)
            number  //= base
        return self.getResult()
    
    def getResult(self):
        sResult      = str()
        while True:
            iElement = self.Rem.pop()
            if iElement is None:
                break
            sResult += self.Charset[iElement]
        return sResult
            

In [9]:
class QR:
    qrDetector = cv2.QRCodeDetector()
    
    def __init__(self, file: str):
        self.pFilename = file
        
    def exists(self) -> bool:
        if (__import__('os').path.isfile(self.pFilename)):
            return True
        return False
    
    def readQR(self):
        if not self.exists():
            return
        
        self.pImage = cv2.imread(self.pFilename)
        return self.__class__.qrDetector.detectAndDecode(self.pImage)
    
    def readStr(self):
        pValue = self.readQR()
        return pValue.__getitem__(0) if pValue is not None else None
        

In [10]:
if __name__ == '__main__':
    cDecimal = Decimal()
    iNumber  = int(PI_1MIL[OFFSET:OFFSET + 6])
    X        = cDecimal.convertTo(iNumber, 26)
    
    QRLINK   = "https://pasteboard.co/{:s}".format(ASCII)
    URL      = QR('qr.png').readStr()
    CONTENT  = BeautifulSoup(requests.get(URL).text)
    Y        = CONTENT.findAll(True, {'class': ['item', 'center']})[0].text
    
    print('Location: {:s}+{:s} Lille'.format(X, Y))

Location: J3M2+M2 Lille
